In [2]:
import Pkg
codepath = joinpath("../../") 
Pkg.activate("mergeSplit"; shared=true)

using Revise
using RandomNumbers
using Random
using MultiScaleMapSampler
using Test
using JSON 
using CSV 

Pkg.develop(path=codepath)


  Activating project at `~/.julia/environments/mergeSplit`
   Resolving package versions...
  No Changes to `~/.julia/environments/mergeSplit/Project.toml`
  No Changes to `~/.julia/environments/mergeSplit/Manifest.toml`


## Load files and graphs

In [3]:
G_json = joinpath("..","test_graphs", "16x16pct.json") # the base graph json must have 1-indexed nodes and edges. 
G_node_data = Set(["county", "pct", "pop", "area", "border_length", "id"])
G_base_graph = BaseGraph(G_json, "pop", inc_node_data=G_node_data,
	area_col="area",node_border_col="border_length", edge_perimeter_col="length")

for k in G_base_graph.node_attributes 
	k["county_and_pct"] = string(k["county"], ",",k["pct"])
end 

G = MultiLevelGraph(G_base_graph, ["county_and_pct"]);
oriented_nbrs = JSON.parsefile("../test_graphs/16x16pct_oriented_nbrs.json")
oriented_nbrs = Dict{GlobalID, Vector{GlobalID}}([parse(Int, k) => v for (k,v) in oriented_nbrs]);

## Generate hierarchy

In [5]:
rng = PCG.PCGStateOneseq(UInt64, 15160)
hierarchy_dir = "../test_graphs/4x4TH_2/tempered_hierarchy_"
TH = TemperingHierarchy(G, oriented_nbrs, hierarchy_dir)
Gs = TH.graphs_by_level;

p1s, p2s, cs = [], [], [] #these are just for logging the different factors considered when creating the hierarchy (e.g. compactness, population)

n_levels = 9
pop_weight1s = [1 for _=1:n_levels]
iso_weights = [0.5 for _=1:n_levels]
amt = [2 for _=1:10]
ideal_nodes_per_level = [16*16]
for i=1:n_levels #in this example, we halve the number of nodes every 2 levels, so a sqrt(2) factor each time we go up a level
    push!(ideal_nodes_per_level, trunc(Int, ideal_nodes_per_level[end] * 2.0^(-1/(amt[i]))))
end
n_by_level = [ideal_nodes_per_level[i] - ideal_nodes_per_level[i+1] for i=1:n_levels]
totalpop = get_node_population(Gs[1], Gs[1].root.global_ID)

for i = 1:n_levels
    @show i, length(finest_level_nodes(Gs[end]))
    n_to_merge = length(finest_level_nodes(Gs[end])) > 2*n_by_level[i] ? n_by_level[i] : length(finest_level_nodes(Gs[end]))/2
    matching = generate_level(TH.graphs_by_level[end], n_to_merge, TH.onbrs_by_level[end], p1s, p2s, cs, pop_weight1s[i], iso_weights[i])
    add_level!(TH, matching) 
end
@show length(finest_level_nodes(Gs[end]))


ideal_nodes_per_level = [256, 215, 180, 142, 100, 70, 49, 34, 24]
n_by_level = [41, 35, 38, 42, 30, 21, 15, 10]
(i, length(finest_level_nodes(Gs[end]))) = (1, 256)
(i, length(finest_level_nodes(Gs[end]))) = (2, 215)
(i, length(finest_level_nodes(Gs[end]))) = (3, 180)
(i, length(finest_level_nodes(Gs[end]))) = (4, 142)
(i, length(finest_level_nodes(Gs[end]))) = (5, 100)
(i, length(finest_level_nodes(Gs[end]))) = (6, 70)
(i, length(finest_level_nodes(Gs[end]))) = (7, 49)
(i, length(finest_level_nodes(Gs[end]))) = (8, 34)
length(finest_level_nodes(Gs[end])) = 24


24